# Introduction to ecnn4klm

`ecnn4klm` can perform large-scale and fast calculations of localized spin dynamics and energy evaluations by substituting the computation of itinerant electrons in the Kondo lattice model. In this tutorial, we will cover data generation, model definition, and training.

## Insatll ecnn4klm

In [ ]:
! pip install git+https://github.com/Miyazaki-Yu/ecnn4klm.git

## Setup

To prepare the training and validation data, we perform exact diagonalization on small lattice sizes ($16\times 16$) to reduce computational costs.

In [ ]:
import torch
import torch.utils.data as data
from ecnn4klm.util.exact import klmsq2d
from ecnn4klm.util.spin_generator import random_spin
from ecnn4klm.model import SpinConvSq2dSparse, NonLinear2d, SelfInt2d
from torch import nn
import e3nn
from e3nn.o3 import Irreps, Irrep
from e3nn import io
import time

In [ ]:
N_train = 100
N_test = 50
L = 16
Lx = L
Ly = L
t = 1.0
J = 7.0
filling = 0.485
device = torch.device("cuda")

In [ ]:
N = N_train + N_test
spin_data = random_spin(N,(Lx,Ly),dtype=torch.float32)
E_data, B_data, _ = klmsq2d(spin_data, t=t, J=J, filling=filling)
spin_train = spin_data[:N_train].to(torch.float32).to(device)
E_train = E_data[:N_train].to(torch.float32).to(device)
B_train = B_data[:N_train].to(torch.float32).to(device)
spin_test = spin_data[N_train:].to(torch.float32).to(device)
E_test = E_data[N_train:].to(torch.float32).to(device)
B_test = B_data[N_train:].to(torch.float32).to(device)
batch_size = 1
train_set = data.TensorDataset(spin_train, E_train, B_train) 
train_loader = data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_set = data.TensorDataset(spin_test, E_test, B_test) 
test_loader = data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)
loss_func = torch.nn.MSELoss()

## Model Definition

`ecnn4klm` is a PyTorch-based library, and neural network models can be defined by inheriting from `nn.Module`. In defining the model, the notation of irreducible representations from the `e3nn` library is used. For more information, please refer to the e3nn website.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l_i = 2
        l_f = 1
        k = 2
        l_hid = 2
        c = 8
        ir_hid = Irreps([(c,(l,1)) for l in range(l_hid+1)])
       
        self.conv1 = SpinConvSq2dSparse(io.SphericalTensor(self.l_i, p_val=1, p_arg=1), l_f, ir_hid, kernel_size=k)
        self.act1 = NonLinear2d(ir_hid, bias = False)
        self.conv2 = SpinConvSq2dSparse(ir_hid, l_f, ir_hid, kernel_size=k)
        self.act2 = NonLinear2d(ir_hid, bias = False)
        self.conv3 = SpinConvSq2dSparse(ir_hid, l_f, ir_hid, kernel_size=k)
        self.act3 = NonLinear2d(ir_hid, bias = False)
        self.conv4 = SpinConvSq2dSparse(ir_hid, l_f, ir_hid, kernel_size=k)
        self.act4 = NonLinear2d(ir_hid, bias = False)  

        self.convout = SpinConvSq2dSparse(ir_hid, l_f, "8x0e", kernel_size=k)      
        
        self.selfintout1 = SelfInt2d("8x0e", "4x0e", biases=True)
        self.actout1 = NonLinear2d("4x0e", bias = False)
        self.selfintout2 = SelfInt2d("4x0e", "2x0e", biases=True)
        self.actout2 = NonLinear2d("2x0e", bias = False)
        self.selfintout3 = SelfInt2d("2x0e", "1x0e", biases=True)



    def forward(self,spin):
        feat = e3nn.o3.spherical_harmonics(list(range(self.l_i+1)), spin, normalize=False)

        feat = self.conv1(feat,spin)
        feat_sk = self.act1(feat)
        feat = self.conv2(feat_sk,spin)
        feat_sk = self.act2(feat+feat_sk)
        feat = self.conv3(feat_sk,spin)
        feat_sk = self.act3(feat+feat_sk)    
        feat = self.conv4(feat_sk,spin)
        feat_sk = self.act4(feat+feat_sk)
 
        feat = self.convout(feat_sk,spin)
        
        feat = self.selfintout1(feat)
        feat = self.actout1(feat)
        feat = self.selfintout2(feat)
        feat = self.actout2(feat)
        feat = self.selfintout3(feat)
        return feat



In [ ]:
net = Net().to(device)
optimizer = torch.optim.Adagrad(net.parameters(), lr=0.1)

## Training

Training of the neural network is performed. With this configuration, it should typically take around 30 minutes using a GPU backend. If you want to further improve the accuracy, try increasing the number of epochs or adjusting various parameters.

In [ ]:
train_losses = []
train_losses_E = [] 
train_losses_B = []
test_losses = []
test_losses_E = []
test_losses_B = []
lamb = 0.0
time_start = time.perf_counter()
best_epoch = 0
for epoch in range(30):
    net.train()
    train_loss = []
    train_loss_E = [] 
    train_loss_B = []
    for spin,E,B in train_loader: 
    
        spin.requires_grad_(True)
        optimizer.zero_grad()

        E_pred = net(spin).sum(dim=(1,2,3))
        B_pred = -torch.autograd.grad(E_pred[0], spin, create_graph=True)[0]
        # B_pred = -torch.autograd.grad(E_pred, spin, torch.eye(batch_size,dtype=torch.float32).to(device), create_graph=True, is_grads_batched=True)[0].sum(dim=1)
        loss_E = loss_func(E,E_pred)/ (Lx*Ly)**2 
        loss_B = loss_func(B,B_pred) 
        loss = loss_B + loss_E * lamb
        train_loss.append(loss.item())
        train_loss_E.append(loss_E.item()) # with E
        train_loss_B.append(loss_B.item())
        
        loss.backward()
        optimizer.step()
    train_losses.append(sum(train_loss)/len(train_loss))
    train_losses_E.append(sum(train_loss_E)/len(train_loss_E)) # with E
    train_losses_B.append(sum(train_loss_B)/len(train_loss_B))

    test_loss = []
    test_loss_E = [] # with E
    test_loss_B = []
    net.eval()
    for spin,E,B in test_loader: # with E
    
        spin.requires_grad_(True)
        E_pred = net(spin).sum(dim=(1,2,3))
        B_pred = -torch.autograd.grad(E_pred[0], spin, create_graph=True)[0]
        # B_pred = -torch.autograd.grad(E_pred, spin, torch.eye(batch_size,dtype=torch.float32).to(device), create_graph=True, is_grads_batched=True)[0].sum(dim=1)
        loss_E = loss_func(E,E_pred)/ (Lx*Ly)**2 # with E
        loss_B = loss_func(B,B_pred)
        loss = loss_B + loss_E * lamb
        test_loss.append(loss.item())
        test_loss_E.append(loss_E.item()) # with E
        test_loss_B.append(loss_B.item())
        
    test_losses.append(sum(test_loss)/len(test_loss))
    test_losses_E.append(sum(test_loss_E)/len(test_loss_E)) # with E
    test_losses_B.append(sum(test_loss_B)/len(test_loss_B))
    time_now = time.perf_counter()

    print(f"epoch {epoch}: lr:{optimizer.param_groups[0]['lr']:.3e}, train:{train_losses[epoch]}, test:{test_losses[epoch]}, elapsed: {time_now-time_start} sec.")